# Lab 5 - Fine-grained Queue Measurement

The monitoring granularity supported by programmable data planes allows them to report statistics on a per-packet basis. In this lab, the user will implement a P4 program that reports the queuing delay to a collector by deploying the mirroring operation. The switch will clone each incoming packet, append the queue occupancy to the cloned instance, and forward it to a collector.  

<img src="./labs_files/queue/figs/topology.png" width="550px"><br>

# Background

## Cloning

Clone operation generates a new version of a packet, allowing the user to process the cloned packet differently from the original packet. The cloning operation does not disturb the ongoing connection as the original packet can be forwarded to its destination, while the cloned packet is directed to the ingress or egress blocks for additional processing. There are four types of cloning: 1) ingress to ingress; 2) ingress to egress; 3) egress to ingress; and 4) egress to egress. The user can specify a list of metadata to be preserved by the cloned packets. Multiple lists can be defined in the metadata data, where the order of the defined lists specify their IDs. Upon cloning, the user can specify which list to use by including it in the cloning function. The order of the lists starts from 1, where ID of value 0 is preserved to the empty list (i.e., no metadata are preserved by the cloned packets). Another parameter of the cloning function is the session ID. The session ID field groups the packets into groups, where different actions can be performed based on the value of this field. 

One function of packet cloning is mirroring. Mirroring (also known as port mirroring) is a standard networking functionality used to send a copy of a packet received on a specific port to a networking monitoring system (e.g., collector) on another port. To implement the mirroring functionality in P4, the user should identify which packets to be monitored, generate the mirrored instances of the identified packets, and specify the actions to be performed on those instances.

Specifying which packets to be monitored is application dependent. For example, suppose the application should report the queue occupancy of the programmable switch on a per-packet basis. In that case, the switch should clone all the incoming packets, append the queue occupancy to the cloned instances, and forward them to a collector. It is important to differentiate between packet cloning and packet mirroring. The term "clone" is used instead of "mirror" to emphasize that it solely generates a new packet version without requiring additional configuration for mirroring. 


## Lab Scenario

In this lab, the user will write a P4 application to report the queue occupancy of the switch on a per-packet basis. The reported values will be forwarded to a collector that will dynamically plot them. To send the measurements to the collector, the program will mirror the incoming packets after appending the queue occupancy to a custom header on the cloned instances. The payload and part of the headers will be discarded from the cloned packets to reduce the overhead of packet processing at the collector. The lab scenario is depicted below.

<img src="./labs_files/queue/figs/scenario.png" width="500px"><br>

# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](./scripts/prepare_env.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [39]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "my_slice"

In [40]:
slice = fablib.new_slice(name="queue2")

In [41]:
#fablib.list_sites()

### Step 3.2: Define the sites
The code below requests four sites from FABRIC: MICH, STAR, IU, and NCSA

<img src="./labs_files/queue/figs/sites.png" width="550px"><br>

In [42]:
#output_table = fablib.list_sites()
#[site1,site2,site3] = fablib.get_random_sites(count=3, avoid=['CERN', 'LOSA', 'FIU', 'NEWY'])
site1='NCSA'
site2='STAR'
site3='MICH'
site4='INDI'

print (f'The selected sites are {site1}, {site2}, {site3}, {site4}') 

The selected sites are NCSA, STAR, MICH, INDI


### Step 3.3: Creating the nodes
The code below creates three nodes: sender, receiver, and collector. The nodes use the following
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

Sender will be created in site1, Receiver will be created in site3, an Collector will be created in site4

<img src="./labs_files/queue/figs/servers.png" width="550px"><br>

In [43]:
sender = slice.add_node(name="sender", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

receiver = slice.add_node(name="receiver", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

collector = slice.add_node(name="collector", 
                      site=site4, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')


### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./labs_files/queue/figs/adding_nics.png" width="550px"><br>

In [44]:
sender_iface = sender.add_component(model='NIC_Basic').get_interfaces()[0]
receiver_iface = receiver.add_component(model='NIC_Basic').get_interfaces()[0]
collector_iface = collector.add_component(model='NIC_Basic').get_interfaces()[0]

### Step 3.5: Creating a node for the P4 switch
The code below creates a node that will run the P4 switch. The node use the following
<ul>
    <li> 16 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 40GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

The node will be created in site2

<img src="./labs_files/queue/figs/adding_switch_noports.png" width="550px"><br>

In [45]:
# Add a node
switch = slice.add_node(name="switch", 
                      site=site2, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

### Step 3.6: Adding two interfaces to the switch
The code below adds three Network Interface Cards (NICs) to the switch.

<img src="./labs_files/queue/figs/adding_switch.png" width="550px"><br>

In [46]:
switch_iface1 = switch.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface2 = switch.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]
switch_iface3 = switch.add_component(model='NIC_Basic', name='net3_nic').get_interfaces()[0]

### Step 3.7: Connecting site1 and site2
Create a site-to-site network between site1 and site2 connecting Sender and the P4 switch

<img src="./labs_files/queue/figs/sender_switch.png" width="550px"><br>

In [47]:
net1 = slice.add_l2network(name='net1', interfaces=[sender_iface, switch_iface1])

### Step 3.7: Connecting site2 and site3
Create a site-to-site network between site2 and site3 connecting the P4 switch and Receiver

<img src="./labs_files/queue/figs/switch_receiver.png" width="550px"><br>

In [48]:
net2 = slice.add_l2network(name='net2', interfaces=[switch_iface2, receiver_iface])

### Step 3.8: Connecting site2 and site4
Create a site-to-site network between site2 and site4 connecting the P4 switch and Collector

<img src="./labs_files/queue/figs/switch_collector.png" width="550px"><br>

In [49]:
net3 = slice.add_l2network(name='net3', interfaces=[switch_iface3, collector_iface])

### Step 3.8: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [50]:
#Submit Slice Request
slice.submit();


Retry: 11, Time: 354 sec


ID,40ca6134-a523-4d93-b954-4859c6dd73a8
Name,queue2
Lease Expiration (UTC),2023-07-28 15:32:10 +0000
Lease Start (UTC),2023-07-27 15:32:10 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
8a8b928d-003d-4995-b371-0c7ba505120c,collector,4,8,100,default_ubuntu_20,qcow2,indi-w3.fabric-testbed.net,INDI,ubuntu,2001:18e8:fff0:3:f816:3eff:feba:3de7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:18e8:fff0:3:f816:3eff:feba:3de7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
47d77405-34f2-4a31-b95a-a9d70d16670c,receiver,4,8,100,default_ubuntu_20,qcow2,mich-w3.fabric-testbed.net,MICH,ubuntu,2607:f018:110:11:f816:3eff:fe07:4341,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2607:f018:110:11:f816:3eff:fe07:4341,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2ac519fe-f903-4173-acf3-2c0d04f05711,sender,4,8,100,default_ubuntu_20,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe92:860f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe92:860f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d1527ed5-b86b-4d5f-b3d5-0f79357608ad,switch,32,16,100,default_ubuntu_20,qcow2,star-w6.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fef4:5da,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fef4:5da,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
bcddd7ca-18db-4ce7-b598-e07c90930bf1,net1,L2,L2STS,None,None,None,Active,
f320a158-ce0a-494e-a2f8-bd9831ecf005,net2,L2,L2STS,None,None,None,Active,
9e22851f-765d-42de-ac48-f66dbeecaf3a,net3,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
sender-None-p1,p1,sender,net1,100,config,,02:FD:CF:9D:C9:F8,ens7,ens7,None,4
receiver-None-p1,p1,receiver,net2,100,config,,1E:C7:24:15:DF:1C,ens7,ens7,None,4
collector-None-p1,p1,collector,net3,100,config,,06:CA:C5:D9:A6:44,ens7,ens7,None,4
switch-net3_nic-p1,p1,switch,net3,100,config,,26:B5:02:38:49:A5,ens9,ens9,None,4
switch-net1_nic-p1,p1,switch,net1,100,config,,12:99:7A:78:6C:25,ens7,ens7,None,4
switch-net2_nic-p1,p1,switch,net2,100,config,,26:94:F7:88:34:1A,ens8,ens8,None,4



Time to print interfaces 361 seconds


# Step 4: Installing the required packages
In this step, we will install the required packages to run the labs. Specifically, we will install the BMv2 software switch and its control plane, the P4 compiler (p4c), and net-tools.


## Step 4.1 Installing BMv2
The BMv2 software switch will be installed on the switch node. We will upload the script [scripts/install_bmv2.sh](./scripts/install_bmv2.sh) to the switch and execute it

In [51]:
switch = slice.get_node(name="switch")     
switch.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.2 Installing net-tools
The net-tools package will be installed on the Sender, Receiver, Collector, and Switch nodes. This package will allow us to use the ifconfig and the arp commands 

In [52]:
sender = slice.get_node(name="sender")
receiver = slice.get_node(name="receiver")
collector = slice.get_node(name="collector")
switch = slice.get_node(name="switch")
stdout, stderr = sender.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = receiver.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = collector.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch.execute(f'sudo apt-get install -y net-tools', quiet=True)

## Step 4.3 Installing scapy
Installing scapy to be able to craft and send raw packets on the servers


In [53]:
stdout, stderr = sender.execute(f'sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = receiver.execute(f'sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = collector.execute(f'sudo apt-get install -y python3-scapy', quiet=True)

# Step 5: Assigning IP and MAC addresses
In this step, we will assign IPv4 addresses to the interfaces of the servers and the switch. We will also hardcode the MAC addresses. 

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/queue/figs/iface_names.png" width="550px"><br>

In [54]:
sender = slice.get_node(name="sender")     
sender_iface = sender.get_interface(network_name='net1') 
sender_iface_name = sender_iface.get_device_name()
print(f'sender_iface: {sender_iface_name}')

receiver = slice.get_node(name="receiver")     
receiver_iface = receiver.get_interface(network_name='net2') 
receiver_iface_name = receiver_iface.get_device_name()
print(f'receiver_iface: {receiver_iface_name}')

collector = slice.get_node(name="collector")     
collector_iface = collector.get_interface(network_name='net3') 
collector_iface_name = collector_iface.get_device_name()
print(f'collector_iface: {collector_iface_name}')

switch = slice.get_node(name="switch")     
switch_iface1 = switch.get_interface(network_name='net1') 
switch_iface1_name = switch_iface1.get_device_name()
print(f'switch_iface1: {switch_iface1_name}')

switch = slice.get_node(name="switch")     
switch_iface2 = switch.get_interface(network_name='net2') 
switch_iface2_name = switch_iface2.get_device_name()
print(f'switch_iface2: {switch_iface2_name}')

switch = slice.get_node(name="switch")     
switch_iface3 = switch.get_interface(network_name='net3') 
switch_iface3_name = switch_iface3.get_device_name()
print(f'switch_iface3: {switch_iface3_name}')

sender_iface: ens7
receiver_iface: ens7
collector_iface: ens7
switch_iface1: ens7
switch_iface2: ens8
switch_iface3: ens9


## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

<img src="./labs_files/queue/figs/ifaces_up.png" width="550px"><br>

In [55]:
stdout, stderr = sender.execute(f'sudo ip link set dev {sender_iface_name} up', quiet=True)
stdout, stderr = receiver.execute(f'sudo ip link set dev {receiver_iface_name} up', quiet=True)
stdout, stderr = collector.execute(f'sudo ip link set dev {collector_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface1_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface2_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface3_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> sender_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>switch_iface1_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
    <li>switch_iface2_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
    <li>switch_iface3_MAC = '00:00:00:00:00:04' (shown as 00:04 in the figure below)</li>
    <li>receiver_iface_MAC = '00:00:00:00:00:05' (shown as 00:05 in the figure below)</li>
    <li>collector_iface_MAC = '00:00:00:00:00:06' (shown as 00:06 in the figure below)</li>
</ul>

<img src="./labs_files/queue/figs/macs.png" width="550px"><br>

In [56]:
sender_iface_MAC = '00:00:00:00:00:01'
switch_iface1_MAC = '00:00:00:00:00:02'
switch_iface2_MAC = '00:00:00:00:00:03'
switch_iface3_MAC = '00:00:00:00:00:04'
receiver_iface_MAC = '00:00:00:00:00:05'
collector_iface_MAC = '00:00:00:00:00:06'

## Step 5.4 Configuring the IP and MAC addresses on server1_iface and switch_iface1

We will use the network 192.168.1.0/24 between Site1 and Site2. We will assign the IP address 192.168.1.10 to server1's interface and 192.168.1.1 to its neighboring interface on the switch.

<img src="./labs_files/queue/figs/sender_switch_ip.png" width="550px"><br>

In [57]:
sender = slice.get_node(name="sender")     

sender_switch_subnet = "192.168.1.0/24"
sender_ip = '192.168.1.10/24'
switch_ip1 = '192.168.1.1/24'

stdout, stderr = sender.execute(f'sudo ifconfig {sender_iface_name} {sender_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} {switch_ip1}')

stdout, stderr = sender.execute(f'sudo ifconfig {sender_iface_name} hw ether {sender_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} hw ether {switch_iface1_MAC}')

## Step 5.5: Configuring the IP and MAC addresses on switch_iface2 and server2_iface

We will use the network 192.168.2.0/24 between Site2 and Site3. We will assign the IP address 192.168.2.10 to server2's interface and 192.168.2.1 to its neighboring interface on the switch.

<img src="./labs_files/queue/figs/rec_switch_ip.png" width="550px"><br>

In [58]:
receiver = slice.get_node(name="receiver")     

receiver_switch_subnet = "192.168.2.0/24"
receiver_ip = '192.168.2.10/24'
switch_ip2 = '192.168.2.1/24'

stdout, stderr = receiver.execute(f'sudo ifconfig {receiver_iface_name} {receiver_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} {switch_ip2}')

stdout, stderr = receiver.execute(f'sudo ifconfig {receiver_iface_name} hw ether {receiver_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} hw ether {switch_iface2_MAC}')

## Step 5.6: Configuring the IP and MAC addresses on switch_iface3 and collector_iface

We will use the network 192.168.3.0/24 between Site2 and Site4. We will assign the IP address 192.168.3.10 to collector's interface and 192.168.3.1 to its neighboring interface on the switch.

<img src="./labs_files/queue/figs/collector_switch_ip.png" width="550px"><br>

In [59]:
collector = slice.get_node(name="collector")     

collector_switch_subnet = "192.168.3.0/24"
collector_ip = '192.168.3.10/24'
switch_ip3 = '192.168.3.1/24'

stdout, stderr = collector.execute(f'sudo ifconfig {collector_iface_name} {collector_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface3_name} {switch_ip3}')

stdout, stderr = collector.execute(f'sudo ifconfig {collector_iface_name} hw ether {collector_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface3_name} hw ether {switch_iface3_MAC}')

# Step 6: Configure forwarding and routing

## Step 6.1: Enable forwarding on the switch

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

In [60]:
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch.execute(command, quiet=True)

## Step 6.2: Delete routing entries for the routes to force traffic to go through the BMv2 switch

In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel

In [61]:
stdout, stderr = switch.execute(f'sudo ip route del {sender_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {receiver_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {collector_switch_subnet}', quiet=True)

## Step 6.3: Configure routing

In this step, we will configure static routes on server1 and server2. 
<ul>
    <li> For Sender, we will add a route to reach the network 192.168.2.0/24 via 192.168.1.1 and the network 192.168.3.0/24 via 192.168.1.1 </li>
    <li> For Receiver, we will add a route to reach the network 192.168.1.0/24 via 192.168.2.1 and the network 192.168.3.0/24 via 192.168.2.1 </li>
    <li> For Collector, we will add a route to reach the network 192.168.1.0/24 via 192.168.3.1 and the network 192.168.2.0/24 via 192.168.3.1 </li>
</ul>

<img src="./labs_files/queue/figs/routing.png" width="550px"><br>

In [62]:
gw1 = switch_ip1.split('/')[0]
gw2 = switch_ip2.split('/')[0]
gw3 = switch_ip3.split('/')[0]
stdout, stderr = sender.execute(f'sudo ip route add {receiver_switch_subnet} via {gw1}')
stdout, stderr = sender.execute(f'sudo ip route add {collector_switch_subnet} via {gw1}')

stdout, stderr = receiver.execute(f'sudo ip route add {sender_switch_subnet} via {gw2}')
stdout, stderr = receiver.execute(f'sudo ip route add {collector_switch_subnet} via {gw2}')

stdout, stderr = collector.execute(f'sudo ip route add {sender_switch_subnet} via {gw3}')
stdout, stderr = collector.execute(f'sudo ip route add {receiver_switch_subnet} via {gw3}')

## Step 6.4: Configure ARP

In this step, we will configure static ARP entries on server1 and server2. The reason we are doing this is because the switch does not process ARP packets unless programmed to. To make sure that ARP packets are not sent towards the switch, we will hardcode the MACs on the servers.

For each server, we will add an ARP entry to its switch's neighboring interface.

In [63]:
stdout, stderr = sender.execute(f'sudo arp -s {gw1} {switch_iface1_MAC}')
stdout, stderr = receiver.execute(f'sudo arp -s {gw2} {switch_iface2_MAC}')
stdout, stderr = collector.execute(f'sudo arp -s {gw3} {switch_iface3_MAC}')

# Step 7: Creating a P4 program to report queue occupancy
   
In this section, you will create a P4 program to report queue occupancy values to the collector. For each incoming packet, the switch will clone it, append the queue occupancy to a custom header on the cloned instance, and forward the cloned instance to the collector.


# Step 7.1: Defining a custom header 

Click on [headers.p4](./labs_files/queue/src/headers.p4) to open the file in the editor.

Define the following custom header by adding the code shown below.

    header queue_t {
        bit<48> queue;
    }


<img src="./labs_files/queue/figs/header_queue.png" width="550px"><br>



<hr>

Append the custom header to current headers by inserting the following line of code. 

    queue_t queue;

<img src="./labs_files/queue/figs/queue_t.png" width="520px"><br>


**Save the changes by pressing Ctrl+s**.

# Step 7.2: Mirroring packets to the collector 

Click on [egress.p4](./labs_files/queue/src/egress.p4) to open the file in the editor.

Add the following code in the egress.p4 file to assign value 2 to PKT_INSTANCE_TYPE_EGRESS_CLONE

    #define PKT_INSTANCE_TYPE_EGRESS_CLONE 2

<img src="./labs_files/queue/figs/define.png" width="570px"><br>

<hr>

Add the following code inside the apply block to check if the current packet is a cloned instance.

    if(standard_metadata.instance_type != PKT_INSTANCE_TYPE_EGRESS_CLONE){
    }

<img src="./labs_files/queue/figs/if.png" width="550px"><br>

In the code above, standard_metadata.instance_type specifies if the packet is an original instance or a cloned instance.  If the instance type is 2, then the instance is a cloned packet. Otherwise, the instance is an original packet. Thus, the if statement checks if the current packet is an original instance.

<hr>

Add the following code inside the if statement to clone original packets using egress to egress cloning.

    clone_preserving_field_list(CloneType.E2E, 8, 0);

<img src="./labs_files/queue/figs/clone_pres.png" width="550px"><br>

The action clone_preserving_field_list has three parameters:
<ul>
    <li> Clone type: this parameter indicates the cloning type (e.g., ingress to egress). </li> 
    <li> Session ID: this parameter indicates the session ID to be attached to the cloned packets. The user defines the mirroring port of the cloned packets using their session IDs. </li>
    <li> ID of the field list: this parameter indicates which list of metadata fields to preserve after cloning the packet. A field list should be defined in the metadata header.</li>
</ul>
  
In the code above, the parameters have the following values:
<ul>
 <li>Clone type is CloneType.E2E indicating that the cloning will be from the egress to the egress.</li>
<li>Session ID is 8 indicating that the cloned packets will have session ID of value 8.</li>
<li>ID of the field list is 0 indicating that no metadata should be preserved by the cloned packets. </li>
</ul>

<hr>

Add the following code to discard the ipv4 and tcp headers of the packets that do not satisfy the if statement (i.e., cloned packets).

    else{
        hdr.ipv4.setInvalid();
        hdr.tcp.setInvalid();
    }

<img src="./labs_files/queue/figs/else_block.png" width="550px"><br>

In the code above, hdr.ipv4.setInvalid() action discards the IPv4 header, and consequently, discards all the data stored inside the header. This reduces the size of the cloned packet, reducing the needed storage at the collector. hdr.tcp.setInvalid() action discards the TCP header.

Note that the Ethernet header is not discarded because it is used to route the packets to the collector. Because the collector and the switch are at the same network, the cloned packets can be routed using the Ethernet header only. However, if the collector was at a different network than the switch, then the IPv4 should be used to route the packet. 

<hr>

Add the following code inside the else statement to set the header hdr.queue to valid and assign the queue depth of the switch to queue field of the header.

    hdr.queue.setValid();
    hdr.queue.queue = (bit<48>)standard_metadata.enq_qdepth;

<img src="./labs_files/queue/figs/set_queue.png" width="550px"><br>

In the code above, the header hdr.queue is set to valid so that the header can be assembled with the packet in the deparser. The queue occupancy is assigned to the queue field of the header (i.e., hdr.queue.queue) using the standard metadata enq_qdepth. Note that standard_metada.enq_qdepth is 19 bits long and it should be cast to 48 bits before being assigned to hdr.queue.queue. This header will be used to report the per-packet occupancy of the switch to the collector.

<hr>

Add the following code inside the else statement to discard the cloned packets’ payload.

    truncate((bit<32>20);

<img src="./labs_files/queue/figs/truncate.png" width="550px"><br>

In the code above, truncate((bit<32>20) leaves the first 20 bytes of the packets and drops everything else. The 20 bytes represent the Ethernet and queue headers as follows: 6 bytes for the source MAC address, 6 bytes for the destination MAC address, 2 bytes to the Ethernet type, and 6 bytes for the queue field of the queue header.

<hr>

Add the following code inside the else statement to modify the Ethernet type of the cloned packets. 

    hdr.ethernet.etherType = 0X1234;

<img src="./labs_files/queue/figs/hdr_ethernet.png" width="550px"><br>

It is necessary to modify the Ethernet type field (hdr.ethernet.etherType) so that the collector can process the cloned packets. The cloned packets have the custom header queue. Because the queue header is after the Ethernet header, the Ethernet Type field should be modified to indicate that the next header is queue and not IPv4 or IPv6. The value of the Ethernet Type field should not be preserved by any protocol (e.g., 0x800 is preserved to IPv4 header and cannot be used).

**Save the changes by pressing Ctrl+s**.

# Step 8: Testing the Program

In this step, we will use the BMv2 switch with logging disabled. By disabling logging, the switch will be able to process traffic at higher rates. 

The binary of the BMv2 switch with logging disabled (*simple_switch_hp*) is located under *scripts/* directory. 

## Step 8.1: Uploading the high performance BMv2 binary to the switch device 
The command below uploads the binary to the switch device.

In [64]:
switch.upload_file('scripts/simple_switch_hp', '/home/ubuntu/simple_switch_hp')
switch.execute('chmod +x /home/ubuntu/simple_switch_hp')

('', '')

## Step 8.2: Uploading the P4 program

The P4 program [basic.p4](labs_files/lab4/src/basic.p4) is located under lab_files/lab3/src.

We will be uploading the whole directory since it includes other P4 files. 

In [65]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/queue/src', '/home/ubuntu/queue')

'success'

## Step 8.3: Disable TCP offloading

The command below disables TCP offloading. This step is crucial to get high rates.

In [66]:
switch.upload_file('scripts/disable_offload.sh', 'disable_offload.sh')
stdout, stderr = switch.execute(f'sudo chmod +x ./disable_offload.sh && sudo ./disable_offload.sh {switch_iface1_name}', quiet=True)
stdout, stderr = switch.execute(f'sudo chmod +x ./disable_offload.sh && sudo ./disable_offload.sh {switch_iface2_name}', quiet=True)
stdout, stderr = switch.execute(f'sudo chmod +x ./disable_offload.sh && sudo ./disable_offload.sh {switch_iface3_name}', quiet=True)

## Step 8.4 Installing iPerf3 and Scapy
iPerf3 will be installed on Sender and Receiver nodes. Scapy will be installed on the collector. We will use the APT package manager for the installation. 

In [67]:
sender = slice.get_node(name="sender")
receiver = slice.get_node(name="receiver")
collector = slice.get_node(name="collector")
stdout, stderr = sender.execute(f'sudo apt-get update && sudo apt-get install -y iperf3', quiet=True)
stdout, stderr = receiver.execute(f'sudo apt-get update && sudo apt-get install -y iperf3', quiet=True)
stdout, stderr = collector.execute('sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)

## Step 8.5: Compiling the P4 program and running the switch

This step will compile the P4 program, then it will stop the previous instance of the switch, will start the high performance switch and allocate the interfaces. 

In [68]:
stdout, stderr = switch.execute(f'sudo p4c queue/src/basic.p4')
stdout, stderr = switch.execute(f'sudo /home/ubuntu/simple_switch_hp -i 0@{switch_iface1_name} -i 1@{switch_iface2_name} -i 2@{switch_iface3_name} basic.json &')

 [--Wwarn=invalid] warning: no user metadata fields tagged with @field_list(0)
 Calling target program-options parser
Adding interface ens7 as port 0
Adding interface ens8 as port 1
Adding interface ens9 as port 2


## Step 8.6: Populating forwarding table from the control plane

In this step we will populate the forwarding table by executing a script. The following rules will be added:

<ul>
    <li>LPM match  : 192.168.1.0/24 => Output port 0"</li>
    <li>LPM match  : 192.168.2.0/24 => Output port 1"</li>
    <li>LPM match  : 192.168.3.0/24 => Output port 2"</li>
</ul>

In [69]:
stdout, stderr = switch.execute('chmod +x queue/src/rules.sh && queue/src/./rules.sh')

Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:01:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:01	00:00
Entry has been added with handle 0
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:02:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:05	00:01
Entry has been added with handle 1
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:03:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:06	00:02
Entry has been added with handle 2
RuntimeCmd: 


## Step 8.7: Configuring queue rate and depth

The first command below sets the queue rate to 1000 packets per second. The Maximum Transmission Unit (MTU) is 1500 bytes/packet. Thus, the sending rate is 1000 packets/second *1500 bytes/packet = 1,500,000 bytes/second = 1.5 Mbytes/second = 12 Mbits/second.

The second command sets the buffer size to 2000 packets (i.e., ~3Mbytes).

The third command sets a mirroring session towards the Collector.

In [70]:
stdout, stderr = switch.execute(f'echo "set_queue_rate 10000" | simple_switch_CLI')
stdout, stderr = switch.execute(f'echo "set_queue_depth 2000" | simple_switch_CLI')
stdout, stderr = switch.execute(f'echo "mirroring_add 8 2" | simple_switch_CLI')

Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: RuntimeCmd: 


## Step 8.8: Starting iPerf3 on server2 

In [71]:
receiver.execute_thread('iperf3 -s')

<Future at 0x7efc380fe370 state=running>

## Step 8.9: Starting the collector process 

The command below pushes the collector python script to the node.

In [72]:
collector.upload_file('labs_files/queue/src/collector.py', '/home/ubuntu/collector.py')
collector.upload_file('labs_files/queue/src/average.sh', '/home/ubuntu/average.sh')
stdout, stderr = collector.execute(f'chmod +x average.sh')

Launch a new terminal by clicking on the "+" button on the top left, then clicking on "Terminal". 

Copy the output of the command below and paste into the terminal to enter to the collector.

In [73]:
collector.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:18e8:fff0:3:f816:3eff:feba:3de7'

Execute the following command in the terminal of the collector

    sudo python3 collector.py | ./average.sh

## Step 8.5: Testing with 50mbit rate 

In this step, we will start an iPerf3 client on the sender and set the rate to 50mbit. Since this rate is smaller than the bottleneck (100mbit), the queue will not be heavily used.

**Execute the cell below and go back to the terminal of the Collector that you opened in the previous step.**

Note that the queue latency is ~ 350us on average. We are averaging every 1000 samples using the ./average.sh script.

In [77]:
sender.execute('iperf3 -c 192.168.2.10 -t 15 -b 50mbit')

Connecting to host 192.168.2.10, port 5201
[  5] local 192.168.1.10 port 48528 connected to 192.168.2.10 port 5201
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  6.03 MBytes  50.6 Mbits/sec    0    206 KBytes       
[  5]   1.00-2.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       
[  5]   2.00-3.00   sec  5.88 MBytes  49.3 Mbits/sec    0    206 KBytes       
[  5]   3.00-4.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       
[  5]   4.00-5.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       
[  5]   5.00-6.00   sec  5.88 MBytes  49.3 Mbits/sec    0    206 KBytes       
[  5]   6.00-7.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       
[  5]   7.00-8.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       
[  5]   8.00-9.00   sec  5.88 MBytes  49.3 Mbits/sec    0    206 KBytes       
[  5]   9.00-10.00  sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       
[  5]  10.00-11.00  sec  6.00 

('Connecting to host 192.168.2.10, port 5201\n[  5] local 192.168.1.10 port 48528 connected to 192.168.2.10 port 5201\n[ ID] Interval           Transfer     Bitrate         Retr  Cwnd\n[  5]   0.00-1.00   sec  6.03 MBytes  50.6 Mbits/sec    0    206 KBytes       \n[  5]   1.00-2.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       \n[  5]   2.00-3.00   sec  5.88 MBytes  49.3 Mbits/sec    0    206 KBytes       \n[  5]   3.00-4.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       \n[  5]   4.00-5.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       \n[  5]   5.00-6.00   sec  5.88 MBytes  49.3 Mbits/sec    0    206 KBytes       \n[  5]   6.00-7.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       \n[  5]   7.00-8.00   sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       \n[  5]   8.00-9.00   sec  5.88 MBytes  49.3 Mbits/sec    0    206 KBytes       \n[  5]   9.00-10.00  sec  6.00 MBytes  50.3 Mbits/sec    0    206 KBytes       \n[  5]  10.00-11

## Step 8.5: Testing without rate limiting

In this step, we will start an iPerf3 client on the sender without setting the rate. Since the rate is unlimited, it is larger than the bottleneck (100mbit), and thus, the queue will heavily used.

**Execute the cell below and go back to the terminal of the Collector that you opened in the previous step.**

Note that the queue latency is ~ 200ms (the maximum latency that the buffer size supports) on average. We are averaging every 1000 samples using the ./average.sh script.

In [78]:
sender.execute('iperf3 -c 192.168.2.10 -t 15')

Connecting to host 192.168.2.10, port 5201
[  5] local 192.168.1.10 port 56714 connected to 192.168.2.10 port 5201
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  16.0 MBytes   134 Mbits/sec    0    868 KBytes       
[  5]   1.00-2.00   sec  13.8 MBytes   115 Mbits/sec    0   1.54 MBytes       
[  5]   2.00-3.00   sec  13.8 MBytes   115 Mbits/sec    0   2.23 MBytes       
[  5]   3.00-4.00   sec  15.0 MBytes   126 Mbits/sec    0   2.92 MBytes       
[  5]   4.00-5.00   sec  13.8 MBytes   115 Mbits/sec  114   1.48 MBytes       
[  5]   5.00-6.00   sec  13.8 MBytes   115 Mbits/sec    0   1.57 MBytes       
[  5]   6.00-7.00   sec  13.8 MBytes   115 Mbits/sec    0   1.64 MBytes       
[  5]   7.00-8.00   sec  13.8 MBytes   115 Mbits/sec    0   1.69 MBytes       
[  5]   8.00-9.00   sec  13.8 MBytes   115 Mbits/sec    0   1.72 MBytes       
[  5]   9.00-10.00  sec  13.8 MBytes   115 Mbits/sec    0   1.74 MBytes       
[  5]  10.00-11.00  sec  13.8 

('Connecting to host 192.168.2.10, port 5201\n[  5] local 192.168.1.10 port 56714 connected to 192.168.2.10 port 5201\n[ ID] Interval           Transfer     Bitrate         Retr  Cwnd\n[  5]   0.00-1.00   sec  16.0 MBytes   134 Mbits/sec    0    868 KBytes       \n[  5]   1.00-2.00   sec  13.8 MBytes   115 Mbits/sec    0   1.54 MBytes       \n[  5]   2.00-3.00   sec  13.8 MBytes   115 Mbits/sec    0   2.23 MBytes       \n[  5]   3.00-4.00   sec  15.0 MBytes   126 Mbits/sec    0   2.92 MBytes       \n[  5]   4.00-5.00   sec  13.8 MBytes   115 Mbits/sec  114   1.48 MBytes       \n[  5]   5.00-6.00   sec  13.8 MBytes   115 Mbits/sec    0   1.57 MBytes       \n[  5]   6.00-7.00   sec  13.8 MBytes   115 Mbits/sec    0   1.64 MBytes       \n[  5]   7.00-8.00   sec  13.8 MBytes   115 Mbits/sec    0   1.69 MBytes       \n[  5]   8.00-9.00   sec  13.8 MBytes   115 Mbits/sec    0   1.72 MBytes       \n[  5]   9.00-10.00  sec  13.8 MBytes   115 Mbits/sec    0   1.74 MBytes       \n[  5]  10.00-11